# Lab3 (Student version)

In [10]:
import random
import matplotlib.pyplot as plt
import sys
import timeit

Download the three following graphs:
- http://lioneltabourier.fr/documents/drosophila.txt
- http://snap.stanford.edu/data/com-LiveJournal.html
- http://snap.stanford.edu/data/email-Eu-core.html (optional, might be too large)

It is also useful to consider some toy graphs (e.g. manually created graphs with a dozen nodes) to test your programs.

## Exercise 0: preliminaries

Using the codes of Lab1, load the graphs in memory as dictionary of lists and check their number of nodes and links.

In [50]:
##CHARGER LE GRAPHE EN MEMOIRE SOUS FORME DE DICTIONNAIRE DE LISTES

def graph_from_file(file_name): #noeud1: [voisin1, voisin2, ...]
    graph = {} #initialiser un dictionnaire vide (graphe vide)
    with open(file_name, "r") as graph_file:
        for line in graph_file:
            if not line[0].isdigit(): #si la ligne ne commence pas par un chiffre
                continue
            node1, node2 = [int(node) for node in line.split()]
            if node1 not in graph:#si le noeud n'est pas dans le graphe
                graph[node1] = [] #ajouter le noeud au graphe et initialiser sa liste de voisins
            graph[node1].append(node2) #ajouter le noeud voisin au noeud
            if node2 not in graph: 
                graph[node2]= [] #pareil pour le noeud 2
            graph[node2].append(node1) 
    return graph

def count_node_link(filename):
    nodes=set() #init un set vide pour suivre les nodes uniques
    
    #init le compteur des nodes et des edges
    count_nodes=0 
    count_edges=0

    with open(filename,"r") as graph_file: #ouvrir le fichier du graphe en mode lecture
        for line in graph_file:
            node1, node2 = [int(node) for node in line.split()] #diviser la ligne en deux nodes et les convertir en int

            
            #verifier si node1 n'est pas dans l'ensemble de nœuds
            if node1 not in nodes:
                count_nodes+=1
                #add le node à l'ensemble des nodes
                nodes.add(node1)
                
            #verifier si node2 n'est pas dans l'ensemble de nœuds
            if node2 not in nodes:
                count_nodes+=1
                #add le node à l'ensemble des nodes
                nodes.add(node2)
            
            count_edges+=1
    
    
    return count_nodes,count_edges #renvoit un tuple (nb nodes, nb edges) pour rcv les deux val dans un seul objet

def compute_degree_dist(graph):
    degree_dist = {}
    for node in graph:
        degree = len(graph[node])
        if degree not in degree_dist:
            degree_dist[degree] = 0
        degree_dist[degree] += 1
    return degree_dist

In [51]:
drosophila_graph=graph_from_file("drosophila.txt") #charge le graphe a partir du fichier .txt dans un dict de lists
print("Nodes, edges: ",count_node_link("drosophila.txt")) #(nodes, edges)

Nodes, edges:  (7236, 22425)


### Que fait ce code?

Deux fonctions distinctes sont définies pour manipuler un graphe stocké dans un fichier. La première fonction, graph_from_file(), charge le graphe à partir d'un fichier en mémoire sous forme d'un dictionnaire de listes, où chaque nœud est associé à une liste de ses voisins. 

La deuxième fonction, count_node_link(), compte le nombre de nœuds et d'arêtes dans le graphe en utilisant un ensemble "nodes" pour suivre les nœuds uniques. 

### Comment?

La fonction graph_from_file() parcourt chaque ligne du fichier du graphe, extrait deux nodes, et les ajoute au dictionnaire graph avec leurs voisins, en initialisant les listes de voisins si besoin.

La fonction count_node_link() utilise un ensemble pour suivre les nœuds uniques, incrémente le compteur de nœuds pour chaque nouveau nœud, et le compteur d'arêtes pour chaque ligne lue. Elle renvoie un tuple avec le nombre de nœuds et d'arêtes.

## Exercise 1: BFS

### 1.1 Components

- Implement a BFS algorithm.  

- Use it on each of the graphs to evaluate the size of the largest connected component of these graphs.
- Use it to identify all connected components.

Warning: if your BFS is not well coded, it can be very long, so if it doesn't work on Amazon or LiveJournal in less than a few minutes, either improve your code, or test only on smaller graphs. 

In [52]:
def bfs(graph, start_node):
    # Initialiser la file avec le node de départ
    queue = [start_node]
    
    # Liste pour suivre les nodes déjà traités
    marked = [start_node]

    # Tant que la file n'est pas vide
    while queue:
        # Retirer le premier nœud de la file
        current_node = queue.pop(0)

        # Parcourir tous les voisins du nœud actuel
        for neighbor in graph[current_node]:
            # Vérifier si le voisin n'a pas encore été visité
            if neighbor not in marked:
                # Ajouter le voisin à la file et à la liste des nodes marqués
                queue.append(neighbor)
                marked.append(neighbor)

    # Renvoyer la liste des nodes visités pendant le BFS
    return marked

#---------------------------------------------------------------#

def compute_size_largest_cnx(graph):
    # Initialisation des structures de données
    nodes_cc_index = {}  # Dictionnaire pour stocker l'index de composant connecté pour chaque nœud
    cc_index = 0  # Index du composant connecté en cours de traitement
    cc_sizes = []  # Liste pour stocker les tailles de tous les composants connectés

    # Initialisation : assigner -1 à chaque nœud pour indiquer qu'il n'a pas encore été traité
    for node in graph:
        nodes_cc_index[node] = -1

    # Parcourir tous les nœuds du graphe
    for node in graph:
        # Si le nœud n'a pas été traité
        if nodes_cc_index[node] == -1:
            # Effectuer un parcours en largeur (BFS) depuis le nœud en cours
            cc = bfs(graph, node)

            # Ajouter la taille du composant connecté à la liste
            cc_sizes.append(len(cc))

            # Assigner l'index du composant connecté à chaque nœud du composant connecté
            for node_marked in cc:
                nodes_cc_index[node_marked] = cc_index

            # Incrémenter l'index du composant connecté pour le prochain composant
            cc_index += 1

    # Retourner la taille du plus grand composant connecté et le dictionnaire d'index
    return max(cc_sizes), nodes_cc_index



In [53]:
graph=graph_from_file("drosophila.txt")
largest_cnx_size, cc_index_mapping = compute_size_largest_cnx(graph)
print("Plus grande composante connexe: ",largest_cnx_size, " nodes ")
print("Composantes connexes pour chaque node ",cc_index_mapping)

Plus grande composante connexe:  7236  nodes 
Composantes connexes pour chaque node  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 



### Fonction BFS

La fonction `bfs(graph, start_node)` effectue un parcours en largeur (BFS) à partir d'un nœud de départ dans un graphe. Elle utilise une file pour explorer les voisins de chaque nœud, marquant les nœuds visités dans la liste `marked`. Cette fonction renvoie la liste des nœuds visités pendant le BFS.

### Fonction compute_size_largest_cnx

La fonction `compute_size_largest_cnx(graph)` utilise le BFS pour calculer la taille du plus grand composant connecté dans un graphe. Elle utilise également un dictionnaire `nodes_cc_index` pour stocker l'index du composant connecté auquel chaque nœud appartient. Les composants connectés sont identifiés en parcourant tous les nœuds du graphe et en attribuant des indices à chaque composant connecté rencontré.

#### Étapes :

1. **Initialisation :**
   - Le dictionnaire `nodes_cc_index` est initialisé avec des valeurs -1 pour indiquer que chaque nœud n'a pas encore été traité.

2. **Parcours des Nœuds :**
   - Chaque nœud du graphe est parcouru.
   - Si un nœud n'a pas été traité (`nodes_cc_index[node] == -1`), un BFS est effectué à partir de ce nœud.

3. **Attribution des Indices :**
   - La taille de chaque composant connecté est ajoutée à la liste `cc_sizes`.
   - Chaque nœud du composant connecté reçoit l'index du composant connecté dans le dictionnaire `nodes_cc_index`.

4. **Résultat :**
   - La fonction renvoie la taille du plus grand composant connecté (`max(cc_sizes)`) et le dictionnaire d'index des composants connectés (`nodes_cc_index`).

Ca fournit des informations sur la connectivité du graphe, en identifiant le plus grand composant connecté et en attribuant des indices à chaque composant connecté trouvé.


### 1.2 Distances

- Modify the BFS above to have it compute the distance to the source node.

- Using the fact that the diameter is necessarily larger than any distance measured, use your distance computation code to get a lower bound of the diameter. The higher the bound, the better.

In [54]:
def compute_distance(graph, node_start):
    # Initialisation de la file (queue) avec le nœud de départ
    queue = [node_start]
    
    # Initialisation du dictionnaire des distances avec des valeurs -1
    distances = {}
    for node in graph:
        distances[node] = -1
    
    # La distance du nœud de départ à lui-même est 0
    distances[node_start] = 0
    
    # Parcours en largeur pour calculer les distances
    while queue:
        node1 = queue.pop(0)
        for node2 in graph[node1]:
            # Si la distance de node2 n'a pas été calculée
            if distances[node2] == -1:
                queue.append(node2)
                # La distance de node2 est la distance de node1 + 1
                distances[node2] = distances[node1] + 1
    
    # Retourner le dictionnaire des distances
    return distances

def compute_diameter(graph, sample_size=10):
    # Sélectionner aléatoirement des nœuds de départ
    nodes_start = random.choices(list(graph.keys()), k=sample_size)
    
    # Calculer les distances maximales pour chaque nœud de départ et retourner le maximum
    return max([max(compute_distance(graph, node_start).values()) for node_start in nodes_start])


In [55]:
print("Distance de chaque node vers le node source:",compute_distance(graph,0))
print("Diamètre",compute_diameter(graph,sample_size=180))

Distance de chaque node vers le node source: {0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 4, 13: 3, 14: 3, 15: 2, 16: 3, 17: 3, 18: 3, 19: 4, 20: 4, 21: 3, 22: 4, 23: 3, 24: 3, 25: 3, 26: 4, 27: 3, 28: 3, 29: 3, 30: 3, 31: 4, 32: 3, 33: 3, 34: 3, 35: 4, 36: 3, 37: 4, 38: 4, 39: 4, 40: 4, 41: 3, 42: 4, 43: 4, 44: 4, 45: 3, 46: 4, 47: 4, 48: 3, 49: 3, 50: 3, 51: 4, 52: 6, 53: 5, 54: 4, 55: 3, 56: 5, 57: 4, 58: 3, 59: 4, 60: 4, 61: 4, 62: 4, 63: 3, 64: 3, 65: 7, 66: 6, 67: 5, 68: 4, 69: 4, 70: 4, 71: 3, 72: 4, 73: 3, 74: 5, 75: 4, 76: 4, 77: 4, 78: 3, 79: 3, 80: 3, 81: 5, 82: 4, 83: 4, 84: 3, 85: 3, 86: 3, 87: 2, 88: 3, 89: 3, 90: 4, 91: 4, 92: 4, 93: 4, 94: 4, 95: 5, 96: 4, 97: 4, 98: 4, 99: 4, 100: 3, 101: 4, 102: 5, 103: 4, 104: 4, 105: 3, 106: 2, 107: 4, 108: 3, 109: 3, 110: 4, 111: 4, 112: 3, 113: 3, 114: 4, 115: 4, 116: 3, 117: 4, 118: 3, 119: 4, 120: 3, 121: 4, 122: 3, 123: 2, 124: 3, 125: 4, 126: 4, 127: 4, 128: 4, 129: 4, 130: 4, 131: 4, 132: 4, 

Diamètre 11


### Calcul de Distance et Diamètre du Graphe

La fonction `compute_distance(graph, node_start)` utilise un parcours en largeur pour calculer les distances entre un nœud de départ et tous les autres nœuds dans le graphe, renvoyant un dictionnaire des distances.

La fonction `compute_diameter(graph, sample_size=10)` sélectionne aléatoirement des nœuds de départ, calcule les distances maximales pour chaque nœud de départ, et retourne le diamètre maximal du graphe.


## Exercise 2: Triangles

### 2.1 Raw triangle counting

- Implement the 2 triangle counting algorithms presented in the course. 

- Test your program on the 3 graphs and report the number of triangles as well as the running time of your program.

In [56]:
def count_triangle(graph):
    # Initialisation du compteur de triangles
    triangle_count = 0
    
    # Parcours de chaque nœud du graphe
    for node1 in graph:
        # Parcours des voisins du nœud actuel
        for node2 in graph[node1]:
            # Vérification pour éviter de compter les triangles deux fois
            if node1 < node2:
                # Parcours des voisins communs entre node1 et node2
                for node3 in graph[node1]:
                    # Vérification de la présence de node3 dans les voisins de node2 et éviter de compter le même triangle plusieurs fois
                    if node3 in graph[node2] and node2 < node3:
                        # Incrémentation du compteur de triangles
                        triangle_count += 1
                        
    # Retourne le nombre total de triangles dans le graphe
    return triangle_count


### Comptage des Triangles dans le Graphe

La fonction `count_triangle(graph)` utilise une approche par force brute pour compter le nombre de triangles dans un graphe non orientée.

1. **Initialisation :** La variable `triangle_count` est initialisée à zéro pour compter le nombre total de triangles.

2. **Parcours des Nœuds :** Pour chaque nœud `node1` dans le graphe :
   - On parcourt les voisins de `node1` (représentés par `node2`).
   - Une vérification est effectuée pour éviter de compter les triangles deux fois, en vérifiant que l'ordre des nœuds est respecté (`node1 < node2`).

3. **Recherche des Voisins Communs :** Pour chaque paire de voisins (`node1`, `node2`), on parcourt les voisins communs (`node3`) de `node1`.
   - On vérifie que `node3` est également un voisin de `node2` et que l'ordre des nœuds est respecté (`node2 < node3`).
   - Si les conditions sont remplies, cela signifie la formation d'un triangle, et le compteur `triangle_count` est incrémenté.

4. **Résultat :** La fonction renvoie le nombre total de triangles dans le graphe, calculé en parcourant tous les nœuds et en vérifiant les relations entre les voisins.


In [57]:
graphe_d=graph_from_file("drosophila.txt")
graphe_e=graph_from_file("email-Eu-core.txt")
graphe_a=graph_from_file("com-amazon.ungraph.txt")

start_time = timeit.default_timer()
print("Nb triangles graphe D: ",count_triangle(graphe_d))

elapsed_time = timeit.default_timer() - start_time
print("Temps d'execution:", elapsed_time, "secondes")

start_time = timeit.default_timer()
print("Nb triangles graphe E: ",count_triangle(graphe_e))

elapsed_time = timeit.default_timer() - start_time
print("Temps d'execution:", elapsed_time, "secondes")

start_time = timeit.default_timer()
print("Nb triangles graphe A: ",count_triangle(graphe_a))

elapsed_time = timeit.default_timer() - start_time
print("Temps d'execution:", elapsed_time, "secondes")


Nb triangles graphe D:  2179
Temps d'execution: 0.08950409991666675 secondes
Nb triangles graphe E:  296095
Temps d'execution: 1.9116909999866039 secondes
Nb triangles graphe A:  667129
Temps d'execution: 1.4894681998994201 secondes


### 2.2 Transitive ratio

Use this program to compute the transitive ratio of the graphs. Remember that the transitive ratio is defined as 
$$ \frac{3.number \ of \ triangles}{number \ of \ forks}$$
and that the number of forks (or connected triples) of a node of degree $d$ is simply $\frac{d(d-1)}{2}$.

In [58]:
def compute_transitive_ratio(graph):
    # Calcul de la distribution des degrés
    degree_dist = compute_degree_dist(graph)
    
    # Initialisation du compteur de triplets
    triple_count = 0
    
    # Parcours de la distribution des degrés
    for degree in degree_dist:
        # Calcul du nombre de triplets associées à chaque degré
        triple_count += degree_dist[degree] * (degree * degree - 1) / 2
    
    # Calcul du ratio transitif en utilisant le nombre de triangles et de fourches
    transitive_ratio = 3 * count_triangle(graph) / triple_count
    
    # Retourne le ratio transitif
    return transitive_ratio


### Calcul du Ratio Transitif dans un Graphe

La fonction `compute_transitive_ratio(graph)` utilise le nombre de triangles et la distribution des degrés pour calculer le ratio transitif d'un graphe non orienté.


1. **Distribution des Degrés :** La fonction commence par calculer la distribution des degrés du graphe en utilisant la fonction `compute_degree_dist(graph)`.

2. **Calcul des Triplets :** Puis elle initialise un compteur `triple_count` et parcourt la distribution des degrés. Pour chaque degré, elle calcule le nombre de triplets associés à ce degré en utilisant la formule `(degree * degree - 1) / 2` et l'ajoute à `triple_count`.

3. **Calcul du Ratio Transitif :** Le nombre total de triangles dans le graphe est obtenu en appelant la fonction `count_triangle(graph)`. Enfin, le ratio transitif est calculé en utilisant la formule `3 * count_triangle(graph) / triple_count`.

4. **Résultat :** La fonction renvoie le ratio transitif du graphe, fournissant une mesure de la connectivité et de la structure des relations dans le réseau.



In [60]:
print("Ratio transitif graphe D: ",compute_transitive_ratio(graphe_d))
print("Ratio transitif graphe E: ",compute_transitive_ratio(graphe_e))
print("Ratio transitif graphe A: ",compute_transitive_ratio(graphe_a))

Ratio transitif graphe D:  0.013550770302316712
Ratio transitif graphe E:  0.2849410308380176
Ratio transitif graphe A:  0.19041557608388043


## * Exercice 3: Recent Research

\* Star exercice is a little bit advanced but very enriching. The point of this exercice is a little introduction to research paper reading and implementing some simple algorithms. 

If you do not do this exercice at all and perfectly do the other 2 (Exercice 1 and 2) you will get a good grade. But doing this exercice even only a part of it will help you get the highest marks.

**I do not want you to understand all the content of the papers. We are just implementing given algorithms. But of course understanding research papers can help you in your future career.**

**Do not hesitate to contact me or email me if you need any help.**

### 3.1  Recent research on triangle counting:

Take a look at the recent article (published in [Alenex 2023](https://epubs.siam.org/doi/book/10.1137/1.9781611977561)
) : https://epubs.siam.org/doi/pdf/10.1137/1.9781611977561.ch7 and implement the algorithms A++ and A+- and compare their running times on the 3 graphs

In [ ]:
#A++
def trouver_triangles(vertex, triangle_courant):
    B[vertex] = True  # Marque le vertex actuel comme visité

    for voisin in graph[vertex]:
        if not B[voisin]:
            # Explore récursivement les voisins
            trouver_triangles(voisin, triangle_courant + [vertex, voisin])

        elif voisin in triangle_courant:
            # Si le voisin fait partie du triangle actuel, cela forme un triangle
            print("Triangle:", triangle_courant + [voisin])

# Traverse chaque vertex et trouve les triangles
for vertex in vertices:
    if not B[vertex]:
        trouver_triangles(vertex, [])


### 3.2  Recent research on Clustering Coefficient:

Take a look at the recent article (published in [LATIN 2022](https://pakal.cs.cinvestav.mx/latin2022/)
) : https://www.inf.ufpr.br/vignatti/v/papers/2022-latin.pdf and implement the algorithm of local clustering coefficient estimation with relative errors of $ \epsilon = 1, 0.5, 0.1$ with probability $ p = 0.25$ and $\delta = 0.8$